In [26]:
import json
from pprint import pprint
import glob
import os
import re
import cv2
import pandas as pd
import numpy as np

word_count = 2
frames = 1
col_names = ['idx','filename']
dfFileName = pd.DataFrame(columns=col_names)
col_names = ['jsonfileidx']

vH_col_names = []
vB_col_names = []


avg_trans_confidence = []
avg_trans_confidence.append([])
avg_trans_confidence.append([])

for i in range(0,21):
    col_names.append("p"+str(i)+"x")
    col_names.append("p"+str(i)+"y")
    col_names.append("p"+str(i)+"c")
    vH_col_names.append("p"+str(i)+"x")
    vH_col_names.append("p"+str(i)+"y")
    vH_col_names.append("p"+str(i)+"c")
    avg_trans_confidence[0].append([])
    avg_trans_confidence[1].append([])

dfLHand = pd.DataFrame(columns = col_names)
dfRHand = pd.DataFrame(columns = col_names)

col_names = ['jsonfileidx']
for i in range(0,25):
    col_names.append("p" + str(i) + "x")
    col_names.append("p" + str(i) + "y")
    col_names.append("p" + str(i) + "c")
    vB_col_names.append("p"+str(i)+"x")
    vB_col_names.append("p"+str(i)+"y")
    vB_col_names.append("p"+str(i)+"c")
dfBodyParts = pd.DataFrame(columns = col_names)

files = glob.glob("/home/yulia/openpose/outputTransition/*.json")
for f in files:
    fName = os.path.basename(f)
    number = re.findall('\d+',f)
    number = max(map(int,number))
    with open(f) as df:
        data = json.load(df)
        lHand = [number]
        rHand = [number]
        body = [number]

        for i in range(0,63):
            lHand.append(data['people'][0]['hand_left_keypoints_2d'][i])
            rHand.append(data['people'][0]['hand_right_keypoints_2d'][i])
            if(i % 3 == 2):
                avg_trans_confidence[0][i//3].append(data['people'][0]['hand_left_keypoints_2d'][i])
                avg_trans_confidence[1][i//3].append(data['people'][0]['hand_right_keypoints_2d'][i])
        for i in range(0,75):
            body.append(data['people'][0]['pose_keypoints_2d'][i])

        dfFileName.loc[len(dfFileName)] = [number,fName]
        dfLHand.loc[len(dfLHand)] = lHand
        dfRHand.loc[len(dfRHand)] = rHand
        dfBodyParts.loc[len(dfBodyParts)] = body

for i in range(0,21):
    avg_trans_confidence[0][i] = sum(avg_trans_confidence[0][i])/len(avg_trans_confidence[0][i])
    avg_trans_confidence[1][i] = sum(avg_trans_confidence[1][i])/len(avg_trans_confidence[1][i])
#     avg_trans_confidence[0][i] = -1
#     avg_trans_confidence[1][i] = -1

arrVids = {}
arrVids[0] = {}
arrVids[0][0] = {}
arrVids[1] = {}
arrVids[1][0] = {}

with open("/home/yulia/openpose/african_american/african_american_000000000060_keypoints.json") as df:
    african_american_lastframe = json.load(df)
arrVids[0][0][0] = african_american_lastframe['people'][0]['hand_left_keypoints_2d']
arrVids[0][0][1] = african_american_lastframe['people'][0]['hand_right_keypoints_2d']
arrVids[0][0][2] = african_american_lastframe['people'][0]['pose_keypoints_2d']

with open("/home/yulia/openpose/advisor/advisor_000000000000_keypoints.json") as df:
    advisor_firstframe = json.load(df)
arrVids[1][0][0] = advisor_firstframe['people'][0]['hand_left_keypoints_2d']
arrVids[1][0][1] = advisor_firstframe['people'][0]['hand_right_keypoints_2d']
arrVids[1][0][2] = advisor_firstframe['people'][0]['pose_keypoints_2d']

# DFs = {}
# for i in range(0,word_count):
#     DFs[i] = {}
#     for j in range(0, frames):
#         DFs[i][j] = {}
#         for k in range (0,3):
#             if(k < 2):
#                 DFs[i][j][k] = pd.DataFrame([arrVids[i][j][k]], columns=vH_col_names)
#             else:
#                 DFs[i][j][k] = pd.DataFrame([arrVids[i][j][k]], columns=vB_col_names)
# len(DFs[0][0][0])
len(arrVids[0][0][0])

63

In [69]:
dfLHand[dfLHand.columns[1]][0] #accessing DF columns by index number

1095.95

In [29]:
arr_conf_index = []
for i in range(0, word_count):
    arr_conf_index.append([])
    for j in range(0,frames):
        arr_conf_index[i].append([])
        for k in range(0, 2):
            arr_conf_index[i][j].append([])
            for m in range(0,len(arrVids[i][j][k]),3):
                if (arrVids[i][j][k][m+2] >= np.mean([arrVids[i][j][k][m+2] for n in range(0,len(arrVids[i][j][k]),3)])):
                    arr_conf_index[i][j][k].append(m//3)

In [30]:
print(arr_conf_index)

[[[[1, 3, 4, 7, 10, 11, 15, 16, 17, 18, 19, 20], [0, 1, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 19, 20]]], [[[2, 3, 4, 5, 6, 7, 8, 9, 11, 14, 15, 16, 17, 18, 19], [0, 1, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 17, 20]]]]


In [32]:
LHand = list(set(arr_conf_index[0][0][0]) & set(arr_conf_index[1][0][0]))
RHand = list(set(arr_conf_index[1][0][1]) & set(arr_conf_index[1][0][1]))
RHand

[0, 1, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 17, 20]

In [58]:
class Points:
    x=[]
    y=[]

In [70]:
predictedPointsL = []
predictedPointsR = []
for i in range(0, 21):
    predictedPointsL.append(Points)
    predictedPointsR.append(Points)
    #left hand
    if(arrVids[0][0][0][i*3] < arrVids[1][0][0][i*3]): 
        predictedPointsL[i].x.append(np.arange(arrVids[0][0][0][i*3], arrVids[1][0][0][i*3], 0.01))
        predictedPointsL[i].y.append(np.arange(arrVids[0][0][0][i*3+1], arrVids[1][0][0][i*3+1], 0.01))
    else:
        predictedPointsL[i].x.append(np.arange(arrVids[0][0][0][i*3], arrVids[1][0][0][i*3], -0.01))
        predictedPointsL[i].y.append(np.arange(arrVids[0][0][0][i*3+1], arrVids[1][0][0][i*3+1], -0.01))
    #right hand
    if(arrVids[0][0][1][i*3] < arrVids[1][0][1][i*3]):
        predictedPointsR[i].x.append(np.arange(arrVids[0][0][1][i*3], arrVids[1][0][1][i*3], 0.01))
        predictedPointsR[i].y.append(np.arange(arrVids[0][0][1][i*3+1], arrVids[1][0][1][i*3+1], 0.01))
    else:
        predictedPointsR[i].x.append(np.arange(arrVids[0][0][1][i*3], arrVids[1][0][1][i*3], -0.01))
        predictedPointsR[i].y.append(np.arange(arrVids[0][0][1][i*3+1], arrVids[1][0][1][i*3+1], -0.01))

In [68]:
distance = []
for i in range(0, len(dfLHand)):
    distance.append([])
    for j in range(0, 21):
        distance[i].append([])
        dist = 0
        for k in range(0, len(predictedPointsL[j].x)):
            dist += np.power(dfLHand[dfLHand.columns[j*3+1]][i]-predictedPointsL[j].x[k],2)+np.power(dfLHand[dfLHand.columns[j*3+2]][i]-predictedPointsL[j].y[k],2)
        for k in range(0, len(predictedPointsR[j].x)):
            dist += np.power(dfRHand)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [16]:
# LHandX = []
# RHandX = []
# LHandY = []
# RHandY = []
# queryL = ""
# queryR = ""
# for i in range(0,len(LHand)):
#     LHandX.append(arrVids[0][0][0][LHand[i]*3])
#     LHandY.append(arrVids[0][0][0][LHand[i]*3+1])
#     LHandX.append(arrVids[1][0][0][LHand[i]*3])
#     LHandY.append(arrVids[1][0][0][LHand[i]*3+1])
#     queryL += str(arrVids[0][0][0][LHand[i]*3])+" <= p"+str(LHand[i])+"x & "+str(arrVids[1][0][0][LHand[i]*3])+" >= p"+str(LHand[i])+"x & "
#     queryL += str(arrVids[0][0][0][LHand[i]*3+1])+" <= p"+str(LHand[i])+"y & "+str(arrVids[1][0][0][LHand[i]*3+1])+" >= p"+str(LHand[i])+"y "
#     if(i < len(LHand)-1):
#         queryL += "& "
# for i in range(0,len(RHand)):
#     RHandX.append(arrVids[0][0][1][RHand[i]*3])
#     RHandY.append(arrVids[0][0][1][RHand[i]*3+1])
#     RHandX.append(arrVids[1][0][1][RHand[i]*3])
#     RHandY.append(arrVids[1][0][1][RHand[i]*3+1])
#     queryR += str(arrVids[0][0][1][RHand[i]*3])+" <= p"+str(RHand[i])+"x & "+str(arrVids[1][0][1][RHand[i]*3])+" >= p"+str(RHand[i])+"x & "
#     queryR += str(arrVids[0][0][1][RHand[i]*3+1])+" <= p"+str(RHand[i])+"y & "+str(arrVids[1][0][1][RHand[i]*3+1])+" >= p"+str(RHand[i])+"y "
#     if(i < len(RHand)-1):
#         queryR += "& "
# queryR

'852.556 <= p0x & 918.065 >= p0x & 627.859 <= p0y & 859.258 >= p0y & 849.163 <= p1x & 940.747 >= p1x & 599.581 <= p1y & 843.057 >= p1y & 862.737 <= p2x & 1007.71 >= p2x & 557.728 <= p2y & 833.336 >= p2y & 888.753 <= p3x & 992.591 >= p3x & 522.662 <= p3y & 793.373 >= p3y & 898.934 <= p4x & 1017.43 >= p4x & 492.121 <= p4y & 792.293 >= p4y & 858.212 <= p5x & 1006.63 >= p5x & 558.859 <= p5y & 824.696 >= p5y & 879.704 <= p6x & 975.31 >= p6x & 537.367 <= p6y & 819.295 >= p6y & 891.016 <= p7x & 956.948 >= p7x & 519.268 <= p7y & 808.495 >= p7y & 897.802 <= p8x & 937.507 >= p8x & 494.383 <= p8y & 810.655 >= p8y & 861.605 <= p9x & 992.591 >= p9x & 581.482 <= p9y & 821.456 >= p9y & 880.835 <= p10x & 979.63 >= p10x & 557.728 <= p10y & 809.575 >= p10y & 891.016 <= p11x & 956.948 >= p11x & 548.679 <= p11y & 804.174 >= p11y & 898.934 <= p12x & 939.667 >= p12x & 539.629 <= p12y & 808.495 >= p12y & 864.999 <= p13x & 1003.39 >= p13x & 600.712 <= p13y & 808.495 >= p13y & 884.229 <= p14x & 977.47 >= p14x 

In [17]:
# LHand

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [18]:
# RHand

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [22]:
arrXL = []
arrYL = []
arrXR = []
arrYR = []

for i in range(0, len(LHand)):
    arrXL.append(LHand[i]*3+1)
    arrYL.append(LHand[i]*3+2)
for j in range(0,len(RHand)):
    arrXR.append(RHand[i]*3+1)
    arrYR.append(RHand[i]*3+2)

dfLHand.iloc[:,arrXL]

,p0x,p1x,p2x,p3x,p4x,p5x,p6x,p7x,p8x,p9x,...,p11x,p12x,p13x,p14x,p15x,p16x,p17x,p18x,p19x,p20x
0,1095.95,1071.570,1043.120,1025.850,1024.830,1038.040,1007.560,988.257,974.034,1037.020,...,982.161,972.002,1036.010,1006.540,987.241,972.002,1036.010,1012.640,1016.700,979.114
1,1097.21,1085.660,1055.190,1029.970,1017.360,1028.920,996.347,970.079,953.267,1022.610,...,961.673,936.456,1024.720,994.245,967.977,946.963,1027.870,1002.650,984.789,969.028
2,1097.32,1082.910,1048.940,1021.140,987.165,1036.580,989.224,970.693,968.633,1029.370,...,961.427,959.368,1029.370,993.342,978.929,974.811,1034.520,1006.730,989.224,979.958
3,1099.13,1093.800,1067.140,1036.220,1040.490,1029.830,998.909,970.122,954.129,1025.560,...,962.659,939.203,1026.630,994.644,969.056,947.732,1027.700,1003.170,985.048,967.990
4,1094.66,1066.240,1029.400,995.717,1033.610,1033.610,1002.030,978.875,954.665,1029.400,...,974.665,953.613,1028.350,1002.030,977.823,953.613,1019.930,1002.030,978.875,959.928
5,1118.87,1092.400,1049.380,1011.880,986.510,1033.940,987.613,962.243,942.389,1038.350,...,961.140,937.977,1043.870,1004.160,975.480,951.213,1048.280,1015.190,995.334,977.686
6,1100.37,1084.270,1055.100,1029.950,1017.880,1039.000,1007.820,981.660,961.541,1038.000,...,979.648,956.511,1039.000,1008.820,987.696,965.565,1044.030,1033.970,1021.900,1011.840
7,1102.56,1074.700,1039.340,1014.700,993.268,1054.340,1028.630,1016.840,1007.200,1050.060,...,1025.410,1024.340,1051.130,1032.910,1023.270,1013.630,1050.060,1031.840,1028.630,1009.340
8,1106.04,1101.650,1097.270,1097.270,1100.560,1104.940,1121.380,1130.150,1141.110,1111.520,...,1138.920,1147.690,1108.230,1117.000,1131.250,1136.730,1108.230,1115.900,1127.960,1132.340
9,1057.08,1034.210,1009.160,916.593,979.759,963.423,922.039,928.573,914.415,967.779,...,955.800,932.929,985.204,956.889,968.868,998.273,1001.540,966.690,983.026,1001.540


In [24]:
dfLHand.query(queryL)
# dfRHand['jsonfileidx'][65]

,jsonfileidx,p0x,p0y,p0c,p1x,p1y,p1c,p2x,p2y,p2c,...,p17c,p18x,p18y,p18c,p19x,p19y,p19c,p20x,p20y,p20c


In [12]:
vidAfricanAmerican = cv2.VideoCapture('/home/yulia/Documents/ASL/african_american.mp4')
cap1 = cv2.VideoCapture('/home/yulia/Documents/transition.mp4')
vidAdvisor = cv2.VideoCapture('/home/yulia/Documents/ASL/advisor.mp4')

# if (cap.isOpened()== False): 
#   print("Error opening video stream or file")
 
# # Read until video is completed
# while(cap.isOpened()):
#   # Capture frame-by-frame
#   ret, frame = cap.read()
#   if ret == True:
 
#     # Display the resulting frame
#     cv2.imshow('Frame',frame)
 
#     # Press Q on keyboard to  exit
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#       break
 
#   # Break the loop
#   else: 
#     break
 
# # When everything done, release the video capture object
# cap.release()
 
# # Closes all the frames
# cv2.destroyAllWindows()

fps = cap1.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
frameCount = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frameCount/fps

print('fps = ' + str(fps))
print('number of frames = ' + str(frameCount))
print('duration (S) = ' + str(duration))
minutes = int(duration/60)
seconds = duration%60
print('duration (M:S) = ' + str(minutes) + ':' + str(seconds))
# cap.release()

cap1.set(1,int(dfRHand['jsonfileidx'][65])/frameCount)
# #The first argument of cap.set(), number 2 defines that parameter for setting the frame selection.
# #Number 2 defines flag CV_CAP_PROP_POS_FRAMES which is a 0-based index of the frame to be decoded/captured next.
# #The second argument defines the frame number in range 0.0-1.0
# cap.set(2,frame_no);

# #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
ret, frame = cap1.read()
height,width,channels = frame.shape
vidAdvisor = cv2.VideoCapture('/home/yulia/Documents/ASL/advisor.mp4')
# #Set grayscale colorspace for the frame. 
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# # #Cut the video extension to have the name of the video
# # my_video_name = video_name.split(".")[0]
# cv2.imshow("abc",frame)
# cv2.waitKey()
# #Display the resulting frame
# # cv2.imshow(my_video_name+' frame '+ str(frame_seq),gray)

# # #Set waitKey 
# # cv2.waitKey()

# # #Store this frame to an image
# # cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'.jpg',gray)

# # # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
# Define the fps to be equal to 10. Also frame size is passed.
 
out = cv2.VideoWriter('/home/yulia/outpy.mp4',cv2.VideoWriter_fourcc('F','M','P', '4'), 10, (width,height))
videofiles = []
videofiles.append(vidAfricanAmerican)
videofiles.append(cap1)
videofiles.append(vidAdvisor)
cap = videofiles[0]
video_index = 0
while(cap.isOpened()):
    if video_index == 1:
        cap.set(cv2.CAP_PROP_POS_FRAMES,int(dfRHand['jsonfileidx'][65])/frameCount)
        video_index +=1
        cap = videofiles[ video_index ]
    ret, frame = cap.read()
    if frame is None:
        print ("end of video " + str(video_index) + " .. next one now")
        video_index += 1
        if video_index >= len(videofiles):
            break
        cap = videofiles[ video_index ]
        ret, frame = cap.read()
    cv2.imshow('frame',frame)
    print(video_index)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

fps = 29.97002997002997
number of frames = 123
duration (S) = 4.1041
duration (M:S) = 0:4.1041
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
end of video 0 .. next one now
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
end of video 2 .. next one now
